In [4]:
#압축파일 풀기
import zipfile
import shutil
import os
folder_path = "./data/pilot_annotated_data/"

zip_ = zipfile.ZipFile(os.path.join(folder_path, "tagtog_League_of_Legend_Relation_Extraction.zip"))
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

FileNotFoundError: [Errno 2] No such file or directory: './data/pilot_annotated_data/tagtog_League_of_Legend_Relation_Extraction.zip'

In [5]:
import json
import glob
import re
import os

pilot_or_main = 'pilot'
#폴더 경로
folder_name = os.path.join(folder_path, 'League_of_Legend_Relation_Extraction/')

#context list
context_name_list = os.listdir(folder_name + "ann.json/master/pool/"+pilot_or_main)

#relation 폴더 경로
relation_folder_paths = [folder_name + "ann.json/master/pool/"+pilot_or_main + "/"+c for c in os.listdir(folder_name + "ann.json/master/pool/"+pilot_or_main)]

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/"+pilot_or_main + "/"+c for c in context_name_list]


#anntation_lenged 정보
annotation_legend = folder_name + "annotations-legend.json"
with open(annotation_legend,"r") as f:
    annotation_legend = json.load(f)
    

In [6]:
context_name_list

['T2258', 'T2077', 'T2046', 'T2196']

In [7]:
relation_folder_paths

['./data/pilot_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/pilot/T2258',
 './data/pilot_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/pilot/T2077',
 './data/pilot_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/pilot/T2046',
 './data/pilot_annotated_data/League_of_Legend_Relation_Extraction/ann.json/master/pool/pilot/T2196']

In [8]:
contexts_folders_paths

['./data/pilot_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/pilot/T2258',
 './data/pilot_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/pilot/T2077',
 './data/pilot_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/pilot/T2046',
 './data/pilot_annotated_data/League_of_Legend_Relation_Extraction/plain.html/pool/pilot/T2196']

In [9]:
#relation dictionary 파일로부터 subject와 object의 entity 정보를 추출해주는 함수
def get_needed_relation_data(tmp_relation):
    subject_token = re.findall("\(+(.+)+\)",annotation_legend[tmp_relation["relations"][0]['classId']])[0].split("|")[0]
    if subject_token == tmp_relation['entities'][0]['classId']:
        sub_entity, obj_entity = tmp_relation['entities']
    else:
        obj_entity, sub_entity  = tmp_relation['entities']
    
    # get preprocessed entities
    def _get_entity(entity):
        outputs = entity['offsets'][0]
        outputs['type'] = annotation_legend[entity['classId']].split("-")[1].lower()
        return outputs
    
    output_subject = _get_entity(sub_entity)
    output_object = _get_entity(obj_entity)
    return output_subject, output_object

In [10]:
#relation dictionary 파일로부터 title(relation)을 추출해주는 함수
def get_label(relation_json):
    label_tag = relation_json['relations'][0]['classId'] #r_6
    try:
        _,sub_type, label = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-")
        return f"{sub_type}:{label}"
    except:
        _,sub_type, = annotation_legend[re.findall("\(+(.+)+\|",annotation_legend[label_tag])[0]].split("-")
        return f"{sub_type}:no_relation"

In [11]:
#html 파일로부터 text만 추출해주는 함수
def get_context_from_html(html_file):
    html_file = re.sub(r"\n","", html_file)
    return re.findall("(<pre.+>)(.+)(</pre>)",html_file)[0][1]

In [12]:
#entity 정보가 포함된 sentence를 생성해주는 함수
def get_sentence_with_entites(subject_entity, object_entity, sentence):
    if subject_entity['start'] < object_entity['start']:
        entity1,entity2 = subject_entity, object_entity
    else:
        entity1,entity2 = object_entity, subject_entity
    
    #entity 시작 위치 및 길이 
    ett1_stt, ett1_len = entity1['start'], len(entity1['text'])
    ett2_stt, ett2_len = entity2['start'], len(entity2['text'])
    
    #문장 분리
    bf, ett1, mid, ett2, af = sentence[:ett1_stt], \
                            sentence[ett1_stt:ett1_stt+ett1_len], \
                            sentence[ett1_stt+ett1_len:ett2_stt], \
                            sentence[ett2_stt:ett2_stt+ett2_len], \
                            sentence[ett2_stt+ett2_len:]

    
    if subject_entity['start'] < object_entity['start']:
        ett1,ett2 = f"<sbj:{ett1}>", f"<obj:{ett2}>"
    else:
        ett1,ett2 = f"<obj:{ett1}>", f"<sbj:{ett2}>"
    
    return "".join([bf, ett1, mid, ett2, af])

In [13]:
#dataframe column
# id : context title(e.g : 카카오게임, 11번가 등)
# sentence w/o entity
# sentence w entity
# subject_entity
# object_entity
# class

id_list = []
sentence_list = []
sentence_with_entities_list = []
subject_entity_list = []
object_entity_list = []
relation_list = []


# tagtog 데이터를 CSV 형태로 변경
for context_name, relation_folder, contexts_folder in zip(context_name_list, relation_folder_paths, contexts_folders_paths):
    # relation files와 context files 리스트 출력
    file_ids = [file_name.split(".ann.")[0] for file_name in os.listdir(relation_folder)]
    file_nums = [ids.split("-")[1] for ids in file_ids]
    relation_files = [relation_folder + "/"+ file_id + ".ann.json" for file_id in file_ids]
    context_files = [contexts_folder + "/"+ file_id + ".plain.html" for file_id in file_ids]
    
    #json으로 된 relation data와 html로 된 context 데이터 읽기
    for relation_file, context_file, file_num in zip(relation_files,context_files, file_nums):
        
        #subject, object 정보 추출
        with open(relation_file, "r") as f:
            relation_json = json.load(f)
        try:
            tmp_subject, tmp_object = get_needed_relation_data(relation_json) #subject, object
            tmp_label = get_label(relation_json)
        except:
            continue
        
        #sentence, sentence with entities 정보 추출
        with open(context_file, "r") as f:
            context_json = f.read()
        tmp_sentence = get_context_from_html(context_json)
        tmp_sentence_w_entities = get_sentence_with_entites(tmp_subject,tmp_object,tmp_sentence)
        
        #각 list에 데이터 저장
        id_list.append(f"{context_name}_{file_num}")
        sentence_list.append(tmp_sentence)
        sentence_with_entities_list.append(tmp_sentence_w_entities)
        subject_entity_list.append(tmp_subject)
        object_entity_list.append(tmp_object)
        relation_list.append(tmp_label.lower())

In [14]:
!pip install gspread
!pip install --upgrade oauth2client

In [18]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './fresh-iridium-332404-8dfc027c47de.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1J9BJVCgzKk6rE29lCdSn_EEZovLeCRTO6DUdIotn6pA/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
## gc.create(spreadsheet_name) # 스프레드시트 생성


#시트 선택하기
sheet_name = pilot_or_main+"_annotation"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

In [21]:
import pandas as pd
# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()
print(values)
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)

column_list = ["id","sentence","sentence_with_entity","subject_entity","object_entity","class"]
data = data[column_list]
data.head()

sen_list =  list(data.sentence_with_entity.values)
len(sen_list)

worksheet.resize(len(id_list)+1,10)
list_range = f"a2:f{len(id_list)+1}"
cell_list = worksheet.range(list_range)

for i in range(len(cell_list)//len(column_list)):
    cell_list[(6*i)].value = id_list[i]
    cell_list[(6*i)+1].value = sentence_list[i]
    cell_list[(6*i)+2].value = sentence_with_entities_list[i]
    cell_list[(6*i)+3].value = str(subject_entity_list[i])
    cell_list[(6*i)+4].value = str(object_entity_list[i])
    cell_list[(6*i)+5].value = relation_list[i]
worksheet.update_cells(cell_list)

[['id', 'sentence', 'sentence_with_entity', 'subject_entity', 'object_entity', 'class']]


{'spreadsheetId': '1J9BJVCgzKk6rE29lCdSn_EEZovLeCRTO6DUdIotn6pA',
 'updatedRange': 'pilot_annotation!A2:F199',
 'updatedRows': 198,
 'updatedColumns': 6,
 'updatedCells': 1188}